# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import requests
import os
import json
#import csv

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

### Overview


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-146485
aml-quickstarts-146485
southcentralus
1b944a9b-fdae-4f97-aeb1-b7eea0beac53


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-heart-failure-prediction'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

experiment

Name,Workspace,Report Page,Docs Page
automl-heart-failure-prediction,quick-starts-ws-146485,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

print(compute_target.get_status().serialize())

Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-06-03T22:23:10.616000+00:00', 'errors': None, 'creationTime': '2021-06-03T22:22:35.815105+00:00', 'modifiedTime': '2021-06-03T22:23:21.224756+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'ST

In [5]:
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction DataSet from UCI"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration



In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure-prediction,AutoML_3d3db5e9-fc29-45d8-a90d-b898fb8179ae,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details



In [8]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure-prediction,AutoML_3d3db5e9-fc29-45d8-a90d-b898fb8179ae,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

*************************************************

{'runId': 'AutoML_3d3db5e9-fc29-45d8-a90d-b898fb8179ae',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-06-03T22:34:27.770402Z',
 'endTimeUtc': '2021-06-03T23:00:11.947376Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-failure-prediction","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-146485","workspace_name":"quick-starts-ws-146485","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000

## Best Model





In [10]:
best_run, fitted_model = remote_run.get_output()
best_metrics = best_run.get_metrics()
print("Best run id : {}".format(best_run.id),
      "Accuracy : {}".format(best_metrics['accuracy']),
      "Best metrics : {}".format(best_metrics),
      "Best model : {}".format(fitted_model), sep = '\n'
     )

Package:azureml-automl-runtime, training version:1.29.0, current version:1.28.0.post2
Package:azureml-core, training version:1.29.0, current version:1.28.0
Package:azureml-dataset-runtime, training version:1.29.0, current version:1.28.0
Package:azureml-defaults, training version:1.29.0, current version:1.28.0
Package:azureml-interpret, training version:1.29.0, current version:1.28.0
Package:azureml-mlflow, training version:1.29.0, current version:1.28.0
Package:azureml-pipeline-core, training version:1.29.0, current version:1.28.0
Package:azureml-telemetry, training version:1.29.0, current version:1.28.0
Package:azureml-train-automl-client, training version:1.29.0, current version:1.28.0
Package:azureml-train-automl-runtime, training version:1.29.0, current version:1.28.0


Best run id : AutoML_3d3db5e9-fc29-45d8-a90d-b898fb8179ae_36
Accuracy : 0.8728735632183909
Best metrics : {'log_loss': 0.37627236413003307, 'average_precision_score_weighted': 0.9251892934172083, 'average_precision_score_macro': 0.9015439261859731, 'matthews_correlation': 0.7183251679637097, 'balanced_accuracy': 0.8417500416885428, 'weighted_accuracy': 0.8925351022387774, 'precision_score_macro': 0.8797016329081547, 'accuracy': 0.8728735632183909, 'f1_score_weighted': 0.8673430085761922, 'recall_score_macro': 0.8417500416885428, 'AUC_micro': 0.9216493592284317, 'precision_score_micro': 0.8728735632183909, 'average_precision_score_micro': 0.9239315729818529, 'AUC_weighted': 0.9140860789985504, 'recall_score_micro': 0.8728735632183909, 'f1_score_macro': 0.84618819916759, 'f1_score_micro': 0.8728735632183909, 'precision_score_weighted': 0.8870201000898152, 'norm_macro_recall': 0.6835000833770858, 'AUC_macro': 0.9140860789985503, 'recall_score_weighted': 0.8728735632183909, 'accuracy_table

In [13]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(
    estimators=[('5', Pipeline(memory=None,
             steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')),
                    ('xgboostclassifier',
                     XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.01, gamma=0, grow_policy='lossguide', max_bin=255, max_depth=6, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
        dataset_samples=299,
        dataset_features=...
        iteration_timeout_param=None,
        feature_column_names=None,
        label_column_name=None,
        weight_column_name=None,
        cv_split_column_names=None,
        enable_streaming=None,
        timeseries_param_dict=None,
        gpu_training_param_dict={'processing_unit_type': 'cpu'}
    ), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='hist'))],
             verbose=False)), ('15', Pipeline(memory=None,
             steps=

In [14]:
#TODO: Save the best model
model = best_run.register_model(model_name = "automl-heart-failure-best-model", model_path = './outputs/model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-146485', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-146485'), name=automl-heart-failure-best-model, id=automl-heart-failure-best-model:1, version=1, tags={}, properties={})


In [14]:
for model in Model.list(ws):
    print(model.name, 'Version: ', model.version)

automl-heart-failure-best-model Version:  1
hyperd-heart-failure-best-model Version:  2
hyperd-heart-failure-best-model Version:  1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
best_run.download_file('/outputs/model.pkl', os.path.join('./model','automl-heart-failure-best-model'))
for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [16]:
model = best_run.register_model(model_name = 'automl-heart-failure-best-model', model_path = './outputs/model.pkl')

In [17]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-environment', file_path = 'conda_env.yml')

In [18]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [19]:
inference_config = InferenceConfig(entry_script = 'score.py', environment = environment)

In [20]:
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights = True)

In [29]:
webservice = Model.deploy(workspace = ws, name = 'heart-failure-service-vs', models=[model], inference_config = inference_config, deployment_config = aci_config, overwrite = True)

In [30]:
webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-03 23:43:57+00:00 Creating Container Registry if not exists.
2021-06-03 23:43:58+00:00 Registering the environment.
2021-06-03 23:44:00+00:00 Building image..
2021-06-03 23:45:45+00:00 Generating deployment configuration.
2021-06-03 23:45:45+00:00 Submitting deployment to compute..
2021-06-03 23:45:48+00:00 Checking the status of deployment heart-failure-service-vs..
2021-06-03 23:48:38+00:00 Checking the status of deployment heart-failure-service-vs..
2021-06-03 23:50:01+00:00 Checking the status of inference endpoint heart-failure-service-vs.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [31]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-146485', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-146485'), name=heart-failure-service-vs, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://0b477168-9fe6-4a94-9b73-0e0d3c6d572f.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})

TODO: In the cell below, send a request to the web service you deployed to test it.

In [39]:
data = {"data": [{"age": 72.0, 
                   "anaemia": 1, 
                   "creatinine_phosphokinase": 110, 
                   "diabetes": 0, "ejection_fraction": 25, 
                   "high_blood_pressure": 0, 
                   "platelets": 274000.0, 
                   "serum_creatinine": 1.0, 
                   "serum_sodium": 140, 
                   "sex": 1, 
                   "smoking": 1, 
                   "time": 65}, 
                  {"age": 56.0, 
                   "anaemia": 1, 
                   "creatinine_phosphokinase": 135, 
                   "diabetes": 1, 
                   "ejection_fraction": 38, 
                   "high_blood_pressure": 0, 
                   "platelets": 133000.0, 
                   "serum_creatinine": 1.7, 
                   "serum_sodium": 140, 
                   "sex": 1, 
                   "smoking": 0, 
                   "time": 244}, 
                 ]}
req_data = json.dumps(data)

In [40]:
scoring_uri = webservice.scoring_uri
scoring_uri

'http://0b477168-9fe6-4a94-9b73-0e0d3c6d572f.southcentralus.azurecontainer.io/score'

In [45]:
headers = {'Content-Type' : 'application/json'}
headers['Authentication'] = f'Bearer '
response = requests.post(scoring_uri, req_data, headers = headers)
print(response.text)['Authentication'] = f'Bearer '

Unauthorized, no token matched


In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

// Request data goes here
data = {
    "data":
    [
        {
            'age': "0",
            'anaemia': "0",
            'creatinine_phosphokinase': "0",
            'diabetes': "0",
            'ejection_fraction': "0",
            'high_blood_pressure': "0",
            'platelets': "0",
            'serum_creatinine': "0",
            'serum_sodium': "0",
            'sex': "0",
            'smoking': "0",
            'time': "0",
        },
    ],
}

body = str.encode(json.dumps(data))

url = 'http://0b477168-9fe6-4a94-9b73-0e0d3c6d572f.southcentralus.azurecontainer.io/score'
api_key = 'bejplwsRsgnrOfSUgDQhJveLASrO9qW7' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

TODO: In the cell below, print the logs of the web service and delete the service

In [43]:
webservice.get_logs()

'2021-06-03T23:49:42,917745000+00:00 - rsyslog/run \n2021-06-03T23:49:42,918211600+00:00 - iot-server/run \n2021-06-03T23:49:42,920154300+00:00 - gunicorn/run \nrsyslogd: /azureml-envs/azureml_9f0c3287c829596c71d31603ecf0cccf/lib/libuuid.so.1: no version information available (required by rsyslogd)\n2021-06-03T23:49:43,000362100+00:00 - nginx/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-06-03T23:49:43,273624300+00:00 - iot-server/finish 1 0\n2021-06-03T23:49:43,275903400+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (66)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 95\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-06-03 23:49:46,016 | root | INFO | Starting up app insights client\n2021-06-03 23:49:46,017 | root | INFO | Starting up request id generato

In [37]:
webservice.delete()